# Machine Learning

This tutorial explains how to train a machine learning model in Safe-DS and use it to make predictions.

## Create a `TaggedTable`

First, we need to create a `TaggedTable` from the training data. `TaggedTable`s are used to train supervised machine learning models, because they keep track of the target
column. A `TaggedTable` can be created from a `Table` by
specifying the target column in the `Table`.

In [ ]:
from safeds.data.tabular.containers import TaggedTable
from safeds.data.tabular.containers import Table

training_set = Table({
    "a":      [3, 4,  8,  6, 5],
    "b":      [2, 2,  1,  6, 3],
    "c":      [1, 1,  1,  1, 1],
    "result": [6, 7, 10, 13, 9]
})

tagged_table = TaggedTable(
    training_set,
    target_column="result"
)

## Create and train model

In this example, we want to predict the column `result`, which is the sum of `a`, `b`, and `c`. We will train a linear regression model with this training data. In Safe-DS, machine learning models are modeled as classes. First, their constructor must be called to configure hyperparameters, which returns a model object. Then, training is started by calling the `fit` method on the model object and passing the training data:

In [ ]:
from safeds.ml.regression import LinearRegression

model = LinearRegression()
model.fit(tagged_table)

## Predicting new values

The `fit` method trains the model in place. This means that the model object is modified and can be used to make predictions. Predictions are made by calling the `predict` method on the model object. The `predict` method takes a `Table` as input and returns a `Table` with the predictions:

In [ ]:
test_set = Table({
    "a": [1, 1, 0, 2, 4],
    "b": [2, 0, 5, 2, 7],
    "c": [1, 4, 3, 2, 1]})

model.predict(dataset=test_set)
